# ACTIVIDAD 3: Deep Learning para Clasificación de Texto 

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

In [1]:
#Basado en:
#https://keras.io/examples/nlp/pretrained_word_embeddings/

#Librerías

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [3]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [4]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['talk.politics.misc', 'rec.autos', 'sci.electronics', 'rec.sport.hockey', 'talk.politics.guns', 'misc.forsale', 'sci.med', 'comp.sys.ibm.pc.hardware', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware', 'rec.sport.baseball', 'rec.motorcycles', 'sci.space', 'soc.religion.christian', 'sci.crypt', 'comp.graphics', 'talk.politics.mideast', 'alt.atheism', 'talk.religion.misc', 'comp.windows.x']


In [5]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['37917', '38287', '37261', '38991', '38782']


In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "39625").read())

Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!newsserver.jvnc.net!castor.hahnemann.edu!hal.hahnemann.edu!brennan
From: brennan@hal.hahnemann.edu
Newsgroups: comp.graphics
Subject: .GIFs on a Tek401x ??
Date: 15 MAY 93 14:29:54 EST
Organization: Hahnemann University
Lines: 14
Message-ID: <15MAY93.14295461@hal.hahnemann.edu>
NNTP-Posting-Host: hal.hahnemann.edu


      I was skimming through a few gophers and bumped into one at NIH
   with a database that included images in .GIF format.  While I have
   not yet worked out the kinks of getting the gopher client to call
   an X viewer, I figure that the majority of the users here are not
   in an X11 environment - instead using DOS and MS-Kermit.

      With Kermit supporting Tek4010 emulation for graphics display,
   does anyone know of a package that would allow a Tek to display a
   .GIF image?  It would be of more use to the local population to
   plug something of this sort in as the 'picture' command in

<font color='green'>**Pregunta 1 (0.5 puntos): Escribe un texto de la categoría 'sci.electronics'**</font>

In [7]:
fnames = os.listdir(data_dir / "sci.electronics")
print("Ejemplo sci.electronics :", fnames[0])
print(open(data_dir / "sci.electronics" / fnames[0]).read())


Ejemplo sci.electronics : 54333
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!pitt.edu!dsinc!ub!acsu.buffalo.edu!ubvmsb.cc.buffalo.edu!v064mb9k
From: v064mb9k@ubvmsb.cc.buffalo.edu (NEIL B. GANDLER)
Newsgroups: sci.electronics
Subject: Looking for a good book on Pspice 5.2
Message-ID: <C65EGz.BG1@acsu.buffalo.edu>
Date: 27 Apr 93 15:19:00 GMT
Sender: nntp@acsu.buffalo.edu
Organization: University at Buffalo
Lines: 6
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsb.cc.buffalo.edu


	I just got a copy of spice 5.2. I would like to know if there are
any published books on the market yet and where I could get one. I would
appreciate any help. Thank You

		Neil Gandler



In [8]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

# Mezclando los datos para separarlos en Traning y Test

In [9]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

<font color='green'>**Pregunta 2 (0.5 puntos): ¿Por qué mezclamos los datos antes de separarlos en entrenamiento y validación?**</font>

Encontramos dos principales motivos:


Facilitar la validación cruzada: La mezcla de datos es esencial para aplicar técnicas como la validación cruzada, asegurando que cada subconjunto de entrenamiento y validación contenga una representación equilibrada y aleatoria de los datos.

Evitar sesgos en la distribución de los datos: Al mezclar los datos, garantizamos que no haya un orden específico que pueda sesgar la distribución de los datos en los conjuntos de entrenamiento.

<font color='green'>**Pregunta 3 (1 punto): ¿Por qué estás seguro que en la 
división aleatoria cada muestra coincide con su etiqueta correcta?**</font>

A traves de la semilla 'seed' se utiliza para generar los instancias separadas de 'np.random.RandomState', asi nos aseguramos que tengan las mismas permutaciones aleatorias tanto para las muestra 'samples' como para las etiquetas 'labels'. Luego en la línea rng.shuffle(samples), se realiza la mezcla aleatoria de las muestras utilizando la primera instancia de RandomState.

El código consigue que cada muestra coincida con su etiqueta correcta después de la división aleatoria.

# Tokenización de las palabras con TextVectorization 

In [10]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [11]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [12]:
len(vectorizer.get_vocabulary())

20000

<font color='green'>**Pregunta 4 (2 puntos): En la construcción del vocabulario hemos limitado el número de tokens a 20.000 ¿Podrías indicar el número de token diferentes o tamaño del vocabulario sin limitar el número de tokens? Es decir, ¿Cuántas palabras diferentes existen en los documentos procesados como instancias?**</font>

In [13]:
# TextVectorization convierte cadenas de texto en listas de tokens y después convierte estas listas de tokens en listas de números
# Crear una instancia de TextVectorization. Sin límite en el número de palabras que se tienen en cuenta. Todas las secuencias de salida se rellenan hasta 200. Si son más largas se truncan.
vectorizer_2 = TextVectorization(max_tokens=None, output_sequence_length=200)

# Objeto Dataset de TensorFlow en batches de 128.
text_ds_2 = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)

# Crear un índice de palabras basado en las palabras que encuentran en los datos de entrenamiento.
# 'adapt' para hacer que el vectorizador "aprenda" el vocabulario de los datos de entrenamiento (contar la frecuencia de cada palabra en los datos y luego asignar un índice a cada palabra)
vectorizer_2.adapt(text_ds_2)

print(len(vectorizer_2.get_vocabulary()))

156962


# Viendo la salida de Vectorizer

In [14]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3550, 1807,   15,    2, 5708])

In [15]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3550, 1807,   15,    2, 5708,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [16]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [17]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3550, 1807, 15, 2, 5708]

<font color='green'>**Pregunta 5 (1 punto): Arriba tenemos la codificación de la frase "the cat sat on the mat". Imagina la siguiente situación. La salida de vectorizer() para codificar los tokens ["El", "gato", "está", "sobre", "el", "tejado"] es la siguiente [1, 121, 405, 1, 45, 4561]. Si cada uno de los valores indica el índice en el que se encuentra cada palabra en el array creado para codificarla. ¿Podría ser correcta esta salida?**</font>

No podria ser correcta, por que despues de convertir a minusculas todas las palabras "El" y "el" tendran que tener el mismo número. Ya que tienen la misma frecuencia.
Está mal en este caso ya que en la primera posición "El" tiene el número 1 y en la quinta posición "el" tiene el 45.

# Tokenización de los datos de entrenamiento y validación

In [18]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación del modelo con un embedding hecho a mano y con redes neuronales convolucionales

Aunque las Redes Neuronales Convolucionales se desarrollaron originalmente para procesamiento de imágenes, se pueden aplicar a problemas de procesamiento de lenguaje natural gracias a su capacidad para capturar patrones locales y globales en los datos de entrada.

Previa a la alimentación de las capas convolucionales, vamos a incluir una capa de Embedding. Como hemos visto en clase, un embedding de palabras se refiere a una representación vectorial densa de una palabra en un espacio de características continuo de alta dimensión. Como resultado del embedding vamos capturar la semántica de las palabras de manera que palabras similares tengan representaciones vectoriales similares.

A continuación se incluye la creación de un modelo con un embedding hecho a mano y usando  Redes Neuronales Convolucionales

In [19]:
modeloEmbeddingManualConvolucionales = keras.models.Sequential()
modeloEmbeddingManualConvolucionales.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.MaxPooling1D(5))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.MaxPooling1D(5))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.GlobalMaxPooling1D())
modeloEmbeddingManualConvolucionales.add(keras.layers.Dense(128, activation='relu'))
modeloEmbeddingManualConvolucionales.add(keras.layers.Dropout(0.3))
modeloEmbeddingManualConvolucionales.add(keras.layers.Dense(20, activation='softmax'))

In [20]:
modeloEmbeddingManualConvolucionales.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManualConvolucionales.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManualConvolucionales.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 24s 152ms/step - loss: 2.9411 - acc: 0.0741 - val_loss: 2.7193 - val_acc: 0.1120
Epoch 2/20
125/125 [==============================] - 13s 107ms/step - loss: 2.5532 - acc: 0.1169 - val_loss: 2.4033 - val_acc: 0.1410
Epoch 3/20
125/125 [==============================] - 13s 106ms/step - loss: 2.2962 - acc: 0.1674 - val_loss: 2.2048 - val_acc: 0.1838
Epoch 4/20
125/125 [==============================] - 8s 61ms/step - loss: 2.1002 - acc: 0.2133 - val_loss: 2.0442 - val_acc: 0.2463
Epoch 5/20
125/125 [==============================] - 8s 68ms/step - loss: 1.9287 - acc: 0.2654 - val_loss: 2.0032 - val_acc: 0.2483
Epoch 6/20
125/125 [==============================] - 6s 46ms/step - loss: 1.7346 - acc: 0.3364 - val_loss: 1.7199 - val_acc: 0.3663
Epoch 7/20
125/125 [==============================] - 6s 46ms/step - loss: 1.5266 - acc: 0.4142 - val_loss: 1.5942 - val_acc: 0.4144
Epoch 8/20
125/125 [==============================] - 5s 38ms/s

<font color='green'>**Pregunta 6 (1 punto): ¿Por qué usamos convoluciones de 1 dimensión para procesar texto?**</font>

<font color='green'>Tu respuesta aqui:

Las convoluciones de 1D son especialmente útiles para extraer características relevantes en datos secuenciales, como el texto, ya que pueden detectar patrones locales en diferentes posiciones dentro de la secuencia.   
Facilita el aprendizaje de características relevantes para tareas de clasificación, generación de texto y otras aplicaciones de procesamiento de lenguaje natural. Estas capas son capaces de capturar patrones locales en secuencias de texto, como frases, n-gramas y combinaciones de palabras adyacentes.

#Creación del modelo con un embedding hecho a mano con redes neuronales clásicas





<font color='green'>**Pregunta 7 (2 puntos): Crea un nuevo modelo partiendo del model anterior pero en lugar de Redes Neuronales Convolucionales vamos a utilizar Redes Neuronales Clásicas. Para ello, tras la capa Embedding añade una capa Flatten, una capa densa de 512 neuronas con función de activación relu y una capa Dropout de regulización al 30%. Por último, no olvides incluir la capa de salida con tantas neuronas como clases haya y la función de activación softmax.**</font>

In [21]:
modeloEmbeddingManualClasicas = keras.models.Sequential()
modeloEmbeddingManualClasicas.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManualClasicas.add(keras.layers.Flatten())
modeloEmbeddingManualClasicas.add(keras.layers.Dense(512, activation='relu'))
modeloEmbeddingManualClasicas.add(keras.layers.Dropout(0.3))
modeloEmbeddingManualClasicas.add(keras.layers.Dense(20, activation='softmax'))

In [22]:
modeloEmbeddingManualClasicas.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManualClasicas.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManualClasicas.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloEmbeddingManualClasicas.summary())

Epoch 1/20
125/125 [==============================] - 15s 111ms/step - loss: 2.8893 - acc: 0.1016 - val_loss: 2.6418 - val_acc: 0.1693
Epoch 2/20
125/125 [==============================] - 11s 88ms/step - loss: 2.2649 - acc: 0.2617 - val_loss: 2.0488 - val_acc: 0.3043
Epoch 3/20
125/125 [==============================] - 10s 81ms/step - loss: 1.6624 - acc: 0.4679 - val_loss: 1.6545 - val_acc: 0.4289
Epoch 4/20
125/125 [==============================] - 6s 47ms/step - loss: 1.1755 - acc: 0.6406 - val_loss: 1.3555 - val_acc: 0.5446
Epoch 5/20
125/125 [==============================] - 7s 52ms/step - loss: 0.8006 - acc: 0.7673 - val_loss: 1.1773 - val_acc: 0.6014
Epoch 6/20
125/125 [==============================] - 3s 21ms/step - loss: 0.5506 - acc: 0.8429 - val_loss: 1.1016 - val_acc: 0.6332
Epoch 7/20
125/125 [==============================] - 3s 27ms/step - loss: 0.3916 - acc: 0.8881 - val_loss: 1.0486 - val_acc: 0.6552
Epoch 8/20
125/125 [==============================] - 5s 39ms/ste

Tras hacer este experimento, es posible que pase una cosa curiosa y es que la red totalmente conectada tenga un mejor comportamiento que la convolucional. 
En general, las redes neuronales convolucionales tienden a superar a las totalmente conectadas en la mayoría de los problemas de este tipo debido a su capacidad para capturar patrones espaciales y posicionales en los datos de entrada. Dicho esto, en algunos casos, una red neuronal clásica podría superar a una convolucional en un problema de procesamiento de lenguaje natural, especialmente si el conjunto de datos es pequeño y las características relevantes son relativamente simples. En este caso, la mayoría de las categorías puede ser que tengan un vocabulario específico y esa pueda ser una de las razones. 





#Creación del modelo con el embedding GloVe y redes neuronales convolucionales


Esta vez, en vez de crear nosotros mismos el embedding, vamos a utilizar Glove. Los vectores de palabras generados por GloVe capturan las relaciones semánticas y sintácticas entre las palabras en un corpus de texto mucho más grande que el nuestro. 

In [23]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-06-06 13:36:44--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-06-06 13:36:44--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-06-06 13:36:45--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

<font color='green'>**Pregunta 8 (1 puntos): 	El embedding que usa la actividad es el glove.6B. ¿De dónde se han obtenido los textos que se han usado para entrenarlo? ¿Cántos WordVectors lo componen? ¿De cuántas dimensiones tiene versiones? Puedes ayudarte del link https://nlp.stanford.edu/projects/glove/ para buscar la información**</font>




<font color='green'>Tu respuesta aqui:

 El corpus de texto utilizado para entrenar los vectores de palabras proviene de Wikipedia 2014 y Gigaword 5, que es un conjunto de noticias en inglés.  
 Consta de 400.000 vectores de palabras únicos.  
 Tiene versiones en 4 dimensiones distintas: 50, 100, 200 y 300.  
 

In [24]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
)
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


Creamos la capa Embedding de keras. Se trata de una matrix numpy donde el valor de cada posición corresponde con el vector pre-entreneado del vocabulario tras el vectorizer

In [25]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18025 words (1975 misses)


Cargamos el embedding como una capa keras. Al poner trainable=False nos quedamos con los valores del modelo pre-entrenado, es decir, no actualizamos estos valores a lo largo del entrenamiento

In [26]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [27]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
modelEmbeddingGloveConvolucionales = keras.Model(int_sequences_input, preds)
modelEmbeddingGloveConvolucionales.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000200   
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [28]:
modelEmbeddingGloveConvolucionales.compile(    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modelEmbeddingGloveConvolucionales.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
predictions = modelEmbeddingGloveConvolucionales.predict(x_val)

Epoch 1/20
125/125 [==============================] - 4s 15ms/step - loss: 2.6758 - acc: 0.1388 - val_loss: 2.1008 - val_acc: 0.2883
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.9680 - acc: 0.3240 - val_loss: 1.5544 - val_acc: 0.4739
Epoch 3/20
125/125 [==============================] - 1s 8ms/step - loss: 1.5272 - acc: 0.4752 - val_loss: 1.2915 - val_acc: 0.5509
Epoch 4/20
125/125 [==============================] - 1s 8ms/step - loss: 1.2786 - acc: 0.5586 - val_loss: 1.1659 - val_acc: 0.5876
Epoch 5/20
125/125 [==============================] - 1s 8ms/step - loss: 1.1282 - acc: 0.6123 - val_loss: 1.1706 - val_acc: 0.6052
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 0.9859 - acc: 0.6572 - val_loss: 1.1390 - val_acc: 0.6067
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 0.8701 - acc: 0.6990 - val_loss: 1.1884 - val_acc: 0.6034
Epoch 8/20
125/125 [==============================] - 1s 9ms/step - loss: 0

#Creación del modelo final


<font color='green'>**Pregunta 9 (1 puntos): 	A estas alturas ya te habrás dado cuenta que los Transformer que has utilizado en la asignatura de Procesamiento de Lenguaje Natural son, probablemente, los que mejores resultados obtienen. Sin utilizar Transformer, crea un modelo tú, bien ajustando los hiperparámetros de los modelos anteriores o uno completamente nuevo, que llegue al menos al 72% accuracy**</font>

In [43]:
from tensorflow.keras import layers, models
 
# Definir la longitud máxima de las secuencias de entrada
sequence_length = x_train.shape[1]
 
# Crear el modelo
model = models.Sequential()
model.add(layers.Embedding(num_tokens, embedding_dim, input_length=sequence_length))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(len(class_names), activation='softmax'))

In [46]:
# Compilar el modelo
model.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
# model.summary()

# Entrenar el modelo
history = model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_val, y_val))

# Obtener las predicciones
predictions = model.predict(x_val)

Epoch 1/20
250/250 [==============================] - 44s 167ms/step - loss: 0.0795 - accuracy: 0.9681 - val_loss: 1.3766 - val_accuracy: 0.6237
Epoch 2/20
250/250 [==============================] - 23s 93ms/step - loss: 0.0645 - accuracy: 0.9699 - val_loss: 1.3655 - val_accuracy: 0.6292
Epoch 3/20
250/250 [==============================] - 14s 54ms/step - loss: 0.0628 - accuracy: 0.9698 - val_loss: 1.3646 - val_accuracy: 0.6327
Epoch 4/20
250/250 [==============================] - 10s 41ms/step - loss: 0.0624 - accuracy: 0.9687 - val_loss: 1.3589 - val_accuracy: 0.6354
Epoch 5/20
250/250 [==============================] - 6s 23ms/step - loss: 0.0647 - accuracy: 0.9685 - val_loss: 1.3421 - val_accuracy: 0.6394
Epoch 6/20
250/250 [==============================] - 7s 30ms/step - loss: 0.0608 - accuracy: 0.9689 - val_loss: 1.3495 - val_accuracy: 0.6374
Epoch 7/20
250/250 [==============================] - 4s 15ms/step - loss: 0.0581 - accuracy: 0.9703 - val_loss: 1.3566 - val_accuracy: 0

In [48]:
model.evaluate(x_val, y_val)

125/125 [==============================] - 0s 2ms/step - loss: 1.3074 - accuracy: 0.6604


[1.307353138923645, 0.6604151129722595]